<a href="https://colab.research.google.com/github/ICSM/ampere/blob/master/docs/source/Ampere_MBB_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Ampere

This notebook introduces the inference package Ampere and demonstrates how to construct a simple model, create some synthetic data from it and the fit the synthetic data with the same model. This is a basic workflow with a few different inference approaches

First we need to import a few key prerequisites

In [ ]:
import numpy as np
import ampere
%matplotlib inline

Ampere splits the problem up into data, model and inference. So the first thing we need to do is define the model we want to use to interpret our data. For this example, we will be using a modified blackbody, one of the simplest yet most widely-used astronomical models

In [ ]:
from ampere.models import Model
from astropy.modeling.physical_models import BlackBody
from astropy import units as u
from scipy.stats import uniform, norm, halfnorm, truncnorm

class ModifiedBlackBody(Model):
    def __init__(self, wavelengths, 
                 kappa=10, kappawave=250,
                 parallax = 10, sigmaparallax = 1,
                 lims = np.array([[10, 50],
                                  [-3,3],
                                  [-3,0],
                                  [0.05,0.15]]),
                 **kwargs):
        self.freq = (wavelengths * u.micron).to(u.Hz, equivalencies=u.spectral()).value
        self.wavelengths = wavelengths
        self.kappa = kappa
        self.kappawave = kappawave
        self.npars = 4
        self.npars_ptform = 4
        self.parallax = parallax
        self.sigmaparallax = sigmaparallax
        self.parallaxsnr = parallax/sigmaparallax
        self.parLabels = ['temperature', 'log10 mass', 'beta', 'distance']
        self.priors = [uniform(lim[0], lim[1]-lim[0]) for lim in lims]

    def __call__(self, t, logm, beta, d, **kwargs):
        modelFlux = BlackBody().evaluate(self.freq, t, 1*u.Jy/u.sr)
        modelFlux = modelFlux/d**2
        modelFlux = modelFlux * (10**(logm))*u.Msun.to(u.g) * (self.kappa) * (self.wavelength/self.kappawave)**beta
        self.modelFlux = modelFlux
        return {"spectrum":{"wavelength":self.wavelengths, "flux": modelFlux}}

    def lnprior(self, theta, **kwargs):
        return np.sum([self.priors[i].logpdf(theta[i]) for i in range(len(theta))])

    def prior_transform(self, u, **kwargs):
        theta = np.array([
                            self.priors[i].ppf(u[i]) for i in range(len(u))
                         ])
        return theta

An Ampere model needs 4 methods. The first of these is the default constructor, `__init__` does any required setup operations. Some of these setup operations are particularly critical for the rest of ampere, such as correctly definining `self.npars` and `self.npars_ptform`, which we will get back to later.

The second, `__call__` controls the behaviour when an instance of this class is called like a function. This method is key, as it is the one that ampere relies upon to predict observables. You will notice that it returns a dictionary containing a spectrum of fluxes. Ampere parses this to produce a results object so that it can more easily convert between different systems.

Ampere models also need to describe the prior for the model parameters. This is *vital* in ampere, since the approach of flexible likelihood functions only makes sense in a Bayesian context - without a prior on the likelihood function we can't get sensible output. Ampere needs you to define the prior in two different methods, reflecting different approaches taken in different inference contexts. The first method, `lnprior`, must return the natural logarithm of the probability density (or mass for discrete variables) function for a given set of input values. That is, given a random sample of model parameters, what is the log of their prior probability? We recommend that this method does indeed return a properly-normalised value, which you can easily achieve by constructing the prior with scipy or torch distributions, however for many cases this only actually needs to be proportional to the prior.

The second prior-related method takes a very different approach. Instead of returning the probability given a sample, it instead seeks to produce samples uniformly distributed under the prior. We therefore want a *bijector* function, which transforms from uniform random variates from the distribution $U[0, 1)$ (i.e. the uniform distribution on the half-open interval $[0, 1)$) to samples drawn from the prior. This type of function is referred to as the *prior transform*. The input to this method is a set of samples from the uniform distribution, one per input to `__call__`. The output is then a random draw from the probability distribution of the values the parameters can take under this prior.

Next we can instantiate an instance of this model and generate some synthetic data. We have to define some true values for the parameters that we're going to use to create the synthetic data, and then later we will try to retrieve these inputs with different inference approaches. 

We also have do define a set of filters in which the synthetic photometry will be calculated. Then we compute the photometry and add some noise before passing that into a `Photometry` object which ampere can understand.

In [ ]:
wavelengths = 10**np.linspace(0.,2.9, 2000)
t_true = 30.
logm_true = -1
beta_true = -2
d_true = 0.11 #kpc

model = ModifiedBlackBody(wavelengths)

res = model(t_true, logm_true, beta_true, d_true)
f_true = res['spectrum']['flux']

import pyphot
#Now we create synthetic photometry
filterName = np.array(['WISE_RSR_W4', 'SPITZER_MIPS_70']) #This is minimal, so we'll just have two bands well separated
#To create synthetic photometry, we need to define the filters we want to use first.
filterName = np.array(['AKARI_FIS_N160', 'AKARI_FIS_N60', 'AKARI_FIS_WIDEL', 'AKARI_FIS_WIDES', 'HERSCHEL_PACS_100', 'HERSCHEL_PACS_160', 'HERSCHEL_PACS_70', 'HERSCHEL_SPIRE_250', 'HERSCHEL_SPIRE_350', 'HERSCHEL_SPIRE_500'])

libDir = ampere.__file__.strip('__init__.py') # '/home/peter/pythonlibs/ampere/ampere/'
libname = libDir + 'ampere_allfilters.hd5'
filterLibrary = pyphot.get_library(fname=libname)
filters = filterLibrary.load_filters(filterName, interp=True, lamb = wavelengths*pyphot.unit['micron'])
filts, modSed = pyphot.extractPhotometry(wavelengths,
                                         f_true,
                                         filters,
                                         Fnu = True,
                                         absFlux = False,
                                         progress=False
        )

input_noise_phot = 0.1 #Fractional uncertainty
photunc = input_noise_phot * modSed #Absolute uncertainty
modSed = modSed + np.random.randn(len(filterName)) * photunc #Now perturb data by drawing from a Gaussian distribution

from ampere.data import Photometry
photometry = Photometry(filterName=filterName, value=modSed, uncertainty=photunc, photUnits='Jy', libName=libname)
#print(photometry.filterMask)
photometry.reloadFilters(wavelengths)
dataset = [photometry]

Now that we have synthetic data, we can set up an inference object and start trying to retrieve our initial parameter values. If you wanted to do this with real data, you could read in the data and pass it to the photometry object instead of creating the synthetic observables. As we will see in other examples, ampere also supports joint inference on photometry and spectroscopy.

We'll start with arguably the simplest inference object in ampere, which is the interface to emcee. Ampere provides interfaces to a range of inference objects, each with different pros and cons. Emcee is simple, familiar and fast, but struggles with some problems (such as when the number of model parameters is large or there are strong correlations between parameters).

In [ ]:
from ampere.infer.emceesearch import EmceeSearch
from emcee import moves

#Ampere exposes the moves interface from emcee, so you can tune how the parameter space gets explored
m = [(moves.DEMove(), 0.8),
    (moves.DESnookerMove(), 0.2),
    ]

optimizer = EmceeSearch(model=model, data=dataset, nwalkers=100, moves=m, vectorize = False)
optimizer.optimise(nsamples = 150, burnin=100, guess='None'
                    )
optimizer.postProcess()

The optimiser here uses emcee to step through the parameter space. First, it uses a minimiser to find an approximate global maximum of the posterior, then initialises the walkers for the ensemble MCMC by randomly perturbing that approximate maximum a posteriori solution. It then begins MCMC sampling with those walkers until it has completed the requested number of steps.

Then it will postprocess the results. By default, this includes computing a few summary statistics, such as the autocorrelation time, Geweke-$z$ statistic and the Rubin-Gelman split $\hat{R}$ statistic for the MCMC chains. It also computes the (approximate) MAP parmaeter values from the MCMC chains, the median and symmetrical 68.3% confidence interval of the parameters' marginal distributions. It also generates some diagnostic plots, including corner plots, the trace and posterior-predictive plots. We can use these to qualitatively examine the output and how successful our inference has been.

Another MCMC package supported by ampere is *zeus*. Zeus uses ensemble slice sampling to explore parameter space. This is generally slower per step than emcee, but it often handles highly-correlated posteriors in fewer steps (such that it may be faster overall) and tends to scale better to larger numbers of parameters. It is particularly useful for multi-modal problems, where the larger diversity of proposals that it can exploit enables it to more effectively transfer walkers between different modes to have a good estimate of their relative probability. The zeus interface follows exactly the same pattern as that for emcee, and produces more-or-less the same outputs.

In [ ]:
from ampere.infer.zeussearch import ZeusSearch
from zeus import moves

#just like emcee, zeus' moves are also exposed to ampere
m = [(moves.GlobalMove(), 0.8),
    (moves.DifferentialMove(), 0.2),
    ]
     
optimizer = ZeusSearch(model=model, data=dataset, nwalkers=100, vectorize = False)
optimizer.optimise(nsamples = 150, burnin=100, guess='None'
                    )
optimizer.postProcess()

Apart from the different choice of sampling approach, zeus produces very similar output to emcee. Since it uses slice sampling, it typically takes longer per accepted sample for simple models, but tends to require many fewer samples for convergence than emcee. 

Otherwise, you get very similar output between these two ensemble-MCMC approaches.

The two previous inference approaches both used MCMC (in different forms) to sample from the posterior. Ampere also supports some very different approaches, including nested sampling with *dynesty*. Nested sampling is a form of constrained-likelihood sampling which can be used to integrate a function by sampling from smaller and smaller bounded regions around the location of the highest value found so far. This is particularly useful for inference, since it enables us to not only explore the shape of the posterior, but also it's normalisation *i.e.* it enables us to estimate the model *evidence*. This makes it an extremely powerful approach when one wants to compare two models for the same observations and assign odds to the models, and it works well even for very complex posteriors with large numbers of parameters. However, for relatively simple models like the one we're using here, it is really overkill if all one wants to do is estimate a credible interval for the parameters.

While the approach is somewhat different, the interface for the user is as similar as possible. The arguments are different, with the main parameter to `optimise()` beyond the point at which to terminate sampling - this is based on the estimate of how much evidence is left unexplored. 

In [ ]:
from ampere.infer.dynestysearch import DynestyNestedSampler
optimizer = DynestyNestedSampler(model=model, data=dataset)
optimizer.optimise(dlogz = 5.)
optimizer.postProcess()

As you can see, the behaviour and output are both rather different from the previous two approaches. While MCMC tends to find the approximate area of greatest probability mass and then sample around that, Nested Sampling instead tried to gradually find regions of higher likelihood, by iteratively replacing the current point with the lowest likelihood until it runs out of regions to explore. As a result, as well as getting samples from the posterior, we get the log-evidence $\ln Z$. If you look through the plots, you will see how the approaches differ. Nevertheless, the final output - a posterior point-estimate and interval - remain more or less the same with these different approaches.

The previous samplers all perform exact inference, albeit with random numbers. However, they rely on sampling the posterior (or sometimes likelihood) of the model directly. For some models this is impossible (e.g. many stochastic models) or infeasible (e.g. very expensive models). In such cases, we may wish to turn to approximate inference - fortunately, ampere supports this too!

Specifically, ampere exploits a type of likelihood-free inference called *neural posterior estimation*, where a shallow neural network is used to directly learn the relationship between the model parameters and the observables the model produces. From this, a posterior is estimated directly, and samples can be generated directly from it without having to use MCMC (or another sampler). This is particularly powerful when your model is very slow to evaluate, or when there are a large number of nuisance parameters, since they are indistinguishable from noise to the estimator. However, like many likelihood-free approximations, it tends to slightly overestimate the width of the posterior if you don't use enough simulations. Nevertheless, when a model is very slow to evaluate, this may be the only way of estimating the posterior within a Hubble time, and it is certaintly better than nothing!

In [ ]:
from ampere.infer.sbi import SBI_SNPE
optimizer = SBI_SNPE(model=model, data=dataset)
optimizer.optimise(nsamples = 10000, nsamples_post = 10000
                   )
optimizer.postProcess()

What did this produce? Let's take a look.